<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/LBN_Dense_AIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [2]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.4 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from sklearn.preprocessing import LabelEncoder
from pgmpy.estimators import HillClimbSearch, BicScore, MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.metrics import accuracy_score
from scipy.stats import entropy
import os
from tabulate import tabulate
from sklearn.model_selection import train_test_split
from pgmpy.estimators import AICScore

# Bayesian Network Data Generation 1000, 2000, ..., 10000 Samples (dense)

In [34]:
# Function to generate CPDs
def generate_cpds():
    # Generate random probabilities for IR
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()  # Normalize to make it a valid probability distribution

    # Generate random probabilities for EI given IR
    ei_given_ir_probs = np.random.rand(3, 3)
    ei_given_ir_probs /= ei_given_ir_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for SP given IR and EI
    sp_probs = np.random.rand(3, 3, 3)
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    return ir_probs, ei_given_ir_probs, sp_probs

# Function to generate and save samples
def generate_and_save_samples(ir_probs, ei_probs, sp_probs, sample_size, filename):
    output_data = []

    # Generate `sample_size` random samples
    for _ in range(sample_size):
        # Sample `IR` state based on `IR` probabilities
        ir_state_idx = np.random.choice(3, p=ir_probs)
        ir_state = ['low', 'medium', 'high'][ir_state_idx]
        ir_prob = ir_probs[ir_state_idx]

        # Sample `EI` state based on `EI` probabilities given `IR`
        ei_probs_given_ir = ei_probs[:, ir_state_idx]
        ei_state_idx = np.random.choice(3, p=ei_probs_given_ir)
        ei_state = ['poor', 'average', 'good'][ei_state_idx]
        ei_prob = ei_probs_given_ir[ei_state_idx]

        # Sample `SP` state based on `SP` probabilities given `IR` and `EI`
        sp_probs_given_ir_ei = sp_probs[:, ir_state_idx, ei_state_idx]
        sp_state_idx = np.random.choice(3, p=sp_probs_given_ir_ei)
        sp_state = ['decrease', 'stable', 'increase'][sp_state_idx]
        sp_prob = sp_probs_given_ir_ei[sp_state_idx]

        # Append sample data to output list
        output_data.append({
            'IR_State': ir_state,
            'IR_Prob': f'{ir_prob:.4f}',
            'EI_State': ei_state,
            'EI_Prob': f'{ei_prob:.4f}',
            'SP_Probabilities (decrease, stable, increase)': ', '.join([f'{prob:.4f}' for prob in sp_probs_given_ir_ei]),
            'Chosen_SP_State': sp_state,
            'Chosen_SP_Probability': f'{sp_prob:.4f}'
        })

    # Create a DataFrame from the output data
    output_df = pd.DataFrame(output_data)

    # Save the output DataFrame to a CSV file
    output_df.to_csv(filename, index=False)

    # Print the first few rows for visual confirmation
    print(f"\nSample size: {sample_size} - First few rows of generated samples:\n")
    print(tabulate(output_df.head(), headers='keys', tablefmt='grid'))

# Generate and save samples for sample sizes from 1000 to 10000 every 1000
sample_sizes = range(1000, 11000, 1000)

for size in sample_sizes:
    # Generate the CPDs
    ir_probs, ei_given_ir_probs, sp_probs = generate_cpds()

    # Generate and save individual samples for the given sample size
    generate_and_save_samples(ir_probs, ei_given_ir_probs, sp_probs, size, f'combined_probabilities_{size}.csv')

# Notify the user that the process is done
print("\nGeneration and saving of individual samples complete for all sample sizes!")


Sample size: 1000 - First few rows of generated samples:

+----+------------+-----------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+
|    | IR_State   |   IR_Prob | EI_State   |   EI_Prob | SP_Probabilities (decrease, stable, increase)   | Chosen_SP_State   |   Chosen_SP_Probability |
+====+============+===========+============+===========+=================================================+===================+=========================+
|  0 | low        |    0.2219 | good       |    0.0666 | 0.1370, 0.1694, 0.6936                          | increase          |                  0.6936 |
+----+------------+-----------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+
|  1 | low        |    0.2219 | poor       |    0.5627 | 0.5340, 0.1117, 0.3543                          | decrease          |                  0.534  |
+----+------------+----

# LBN Dense AIC

In [35]:
# Sample sizes to loop through
sample_sizes = range(1000, 11000, 1000)

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size}")

    # Load the dense dataset for the current sample size
    dense_data_file = f'combined_probabilities_{sample_size}.csv'
    df_dense = pd.read_csv(dense_data_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df_dense['IR_encoded'] = df_dense['IR_State'].map(ir_map)
    df_dense['EI_encoded'] = df_dense['EI_State'].map(ei_map)
    df_dense['SP_encoded'] = df_dense['Chosen_SP_State'].map(sp_map)

    # Split the data into training, validation, and test sets
    X = df_dense[['IR_encoded', 'EI_encoded']]
    y = df_dense['SP_encoded']

    # Split into training (70%) and temp (30%) for validation and test
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
    # Split temp into validation (50%) and test (50%)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)

    # Concatenate X and y to form the training set for learning the BN structure
    df_train = pd.concat([X_train, y_train], axis=1)

    # Define the Hill-Climb structure learning algorithm
    hc = HillClimbSearch(df_train)
    scoring_method = AICScore(df_train)  # <-- Change from BicScore to AicScore

    # Estimate the best structure
    best_dag = hc.estimate(scoring_method=scoring_method)
    best_model = BayesianNetwork(best_dag.edges())

    # Display the learned structure (edges of the Bayesian Network)
    print(f"\nLearned Structure (Edges) for {sample_size} samples:")
    print(best_model.edges())

    # Learn the CPDs using Maximum Likelihood Estimation (MLE)
    best_model.fit(df_train, estimator=MaximumLikelihoodEstimator)

    # Check if the model is valid after learning the parameters
    assert best_model.check_model()

    # Print the learned CPDs (Conditional Probability Distributions)
    for cpd in best_model.get_cpds():
        print("\nCPD of", cpd.variable)
        print(cpd)

print("\nProcessing complete for all sample sizes.")


Processing sample size: 1000


  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 1000 samples:
[('EI_encoded', 'IR_encoded'), ('EI_encoded', 'SP_encoded'), ('SP_encoded', 'IR_encoded')]

CPD of EI_encoded
+---------------+----------+
| EI_encoded(0) | 0.425714 |
+---------------+----------+
| EI_encoded(1) | 0.147143 |
+---------------+----------+
| EI_encoded(2) | 0.427143 |
+---------------+----------+

CPD of IR_encoded
+---------------+--------------------+-----+----------------------+---------------+
| EI_encoded    | EI_encoded(0)      | ... | EI_encoded(2)        | EI_encoded(2) |
+---------------+--------------------+-----+----------------------+---------------+
| SP_encoded    | SP_encoded(0)      | ... | SP_encoded(1)        | SP_encoded(2) |
+---------------+--------------------+-----+----------------------+---------------+
| IR_encoded(0) | 0.5471698113207547 | ... | 0.014285714285714285 | 0.15          |
+---------------+--------------------+-----+----------------------+---------------+
| IR_encoded(1) | 0.452830188679245

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 2000 samples:
[('IR_encoded', 'SP_encoded'), ('IR_encoded', 'EI_encoded'), ('EI_encoded', 'SP_encoded')]

CPD of IR_encoded
+---------------+----------+
| IR_encoded(0) | 0.519286 |
+---------------+----------+
| IR_encoded(1) | 0.174286 |
+---------------+----------+
| IR_encoded(2) | 0.306429 |
+---------------+----------+

CPD of SP_encoded
+---------------+---------------------+-----+--------------------+---------------------+
| EI_encoded    | EI_encoded(0)       | ... | EI_encoded(2)      | EI_encoded(2)       |
+---------------+---------------------+-----+--------------------+---------------------+
| IR_encoded    | IR_encoded(0)       | ... | IR_encoded(1)      | IR_encoded(2)       |
+---------------+---------------------+-----+--------------------+---------------------+
| SP_encoded(0) | 0.06666666666666667 | ... | 0.0743801652892562 | 0.3228346456692913  |
+---------------+---------------------+-----+--------------------+---------------------+


...


Learned Structure (Edges) for 8000 samples:
[('IR_encoded', 'EI_encoded'), ('IR_encoded', 'SP_encoded'), ('SP_encoded', 'EI_encoded')]

CPD of IR_encoded
+---------------+----------+
| IR_encoded(0) | 0.320536 |
+---------------+----------+
| IR_encoded(1) | 0.245179 |
+---------------+----------+
| IR_encoded(2) | 0.434286 |
+---------------+----------+

CPD of EI_encoded
+---------------+---------------------+-----+---------------------+--------------------+
| IR_encoded    | IR_encoded(0)       | ... | IR_encoded(2)       | IR_encoded(2)      |
+---------------+---------------------+-----+---------------------+--------------------+
| SP_encoded    | SP_encoded(0)       | ... | SP_encoded(1)       | SP_encoded(2)      |
+---------------+---------------------+-----+---------------------+--------------------+
| EI_encoded(0) | 0.39896373056994816 | ... | 0.6450892857142857  | 0.2870292887029289 |
+---------------+---------------------+-----+---------------------+--------------------+


  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 9000 samples:
[('IR_encoded', 'EI_encoded'), ('SP_encoded', 'EI_encoded'), ('SP_encoded', 'IR_encoded')]

CPD of IR_encoded
+---------------+----------------------+---------------------+----------------------+
| SP_encoded    | SP_encoded(0)        | SP_encoded(1)       | SP_encoded(2)        |
+---------------+----------------------+---------------------+----------------------+
| IR_encoded(0) | 0.012372013651877133 | 0.01751592356687898 | 0.016296296296296295 |
+---------------+----------------------+---------------------+----------------------+
| IR_encoded(1) | 0.8080204778156996   | 0.6202229299363057  | 0.6214814814814815   |
+---------------+----------------------+---------------------+----------------------+
| IR_encoded(2) | 0.17960750853242322  | 0.3622611464968153  | 0.3622222222222222   |
+---------------+----------------------+---------------------+----------------------+

CPD of EI_encoded
+---------------+--------------------+-----+--------

  0%|          | 0/1000000 [00:00<?, ?it/s]


Learned Structure (Edges) for 10000 samples:
[('IR_encoded', 'SP_encoded'), ('IR_encoded', 'EI_encoded'), ('EI_encoded', 'SP_encoded')]

CPD of IR_encoded
+---------------+-----------+
| IR_encoded(0) | 0.137571  |
+---------------+-----------+
| IR_encoded(1) | 0.0852857 |
+---------------+-----------+
| IR_encoded(2) | 0.777143  |
+---------------+-----------+

CPD of SP_encoded
+---------------+--------------------+-----+---------------------+---------------------+
| EI_encoded    | EI_encoded(0)      | ... | EI_encoded(2)       | EI_encoded(2)       |
+---------------+--------------------+-----+---------------------+---------------------+
| IR_encoded    | IR_encoded(0)      | ... | IR_encoded(1)       | IR_encoded(2)       |
+---------------+--------------------+-----+---------------------+---------------------+
| SP_encoded(0) | 0.5974025974025974 | ... | 0.0                 | 0.4302096985583224  |
+---------------+--------------------+-----+---------------------+---------------

# Entropy

In [36]:
inference = VariableElimination(best_model)

# Sample sizes to loop through
sample_sizes = range(1000, 11000, 1000)

# Prepare a list to store K-L divergence results
kl_divergence_results = []

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing K-L Divergence for sample size: {sample_size}")

    # Load the dense dataset used in the LBN part
    dense_data_file = f'combined_probabilities_{sample_size}.csv'
    df_dense = pd.read_csv(dense_data_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df_dense['IR_encoded'] = df_dense['IR_State'].map(ir_map)
    df_dense['EI_encoded'] = df_dense['EI_State'].map(ei_map)
    df_dense['SP_encoded'] = df_dense['Chosen_SP_State'].map(sp_map)

    # Use the test data split obtained from the LBN part
    X_test = df_dense[['IR_encoded', 'EI_encoded']]
    y_test = df_dense['SP_encoded']

    # Placeholder to store K-L divergence values
    kl_divergences = []

    # Loop through each row in the test data to make predictions
    for index, row in X_test.iterrows():
        sample_input = {'IR_encoded': int(row['IR_encoded']), 'EI_encoded': int(row['EI_encoded'])}

        # Perform inference using the learned Bayesian model
        predicted_sp_distribution = inference.query(variables=['SP_encoded'], evidence=sample_input)
        predicted_probs = predicted_sp_distribution.values

        # Extract the ground truth probabilities for SP from `y_test`
        ground_truth_probabilities_str = df_dense['SP_Probabilities (decrease, stable, increase)'].iloc[index]
        ground_truth_probs = np.array(list(map(float, ground_truth_probabilities_str.strip('[]').split(','))))

        # Ensure the probabilities are non-zero to avoid division by zero
        epsilon = 1e-10
        ground_truth_probs = np.clip(ground_truth_probs, epsilon, 1)
        predicted_probs = np.clip(predicted_probs, epsilon, 1)

        # Normalize both probability distributions
        ground_truth_probs /= ground_truth_probs.sum()
        predicted_probs /= predicted_probs.sum()

        # Calculate the K-L divergence (Learned BN vs Ground Truth)
        kl_div = entropy(ground_truth_probs, predicted_probs)
        kl_divergences.append(kl_div)

    # Calculate the average K-L divergence and standard deviation over all test samples
    average_kl_divergence = np.mean(kl_divergences)
    std_kl_divergence = np.std(kl_divergences)

    # Append the results to the list for saving later
    kl_divergence_results.append({
        'Sample_Size': sample_size,
        'Average_KL_Divergence': average_kl_divergence,
        'Std_Dev': std_kl_divergence
    })

    # Print confirmation and result for this sample size
    print(f"\nAverage K-L Divergence for {sample_size} samples: {average_kl_divergence:.4f}, Std Dev: {std_kl_divergence:.4f}")

# Save the K-L divergence results to a CSV file
kl_divergence_df = pd.DataFrame(kl_divergence_results)
kl_divergence_df.to_csv('kl_div_LBN_dense_aic.csv', index=False)  # <-- Changed the filename to reflect AIC

print("\nK-L divergence calculations complete and results saved to 'kl_div_LBN_dense_aic.csv'.")


Processing K-L Divergence for sample size: 1000

Average K-L Divergence for 1000 samples: 3.1461, Std Dev: 3.4999

Processing K-L Divergence for sample size: 2000

Average K-L Divergence for 2000 samples: 0.2097, Std Dev: 0.3249

Processing K-L Divergence for sample size: 3000

Average K-L Divergence for 3000 samples: 2.4070, Std Dev: 4.2321

Processing K-L Divergence for sample size: 4000

Average K-L Divergence for 4000 samples: 0.6601, Std Dev: 1.5017

Processing K-L Divergence for sample size: 5000

Average K-L Divergence for 5000 samples: 0.8588, Std Dev: 1.2247

Processing K-L Divergence for sample size: 6000

Average K-L Divergence for 6000 samples: 0.9245, Std Dev: 2.4628

Processing K-L Divergence for sample size: 7000

Average K-L Divergence for 7000 samples: 1.1994, Std Dev: 1.2842

Processing K-L Divergence for sample size: 8000

Average K-L Divergence for 8000 samples: 1.4600, Std Dev: 2.2643

Processing K-L Divergence for sample size: 9000

Average K-L Divergence for 900